<a href="https://colab.research.google.com/github/alfanugraha/rumahcoding/blob/master/machine-learning-mini-bootcamp/model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install Library

In [ ]:
!pip install pandas streamlit babel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.9 MB/s eta 0:00:00


## 2. Persiapan Model

### a. Download Model dan Scaler yang Sudah diTraining

In [ ]:
!gdown 1h1560ZLBqX4G3Rnpd-RtcDJUne_pnbwA

Downloading...
From: https://drive.google.com/uc?id=1h1560ZLBqX4G3Rnpd-RtcDJUne_pnbwA
To: /content/models.zip
100% 5.33M/5.33M [00:00<00:00, 133MB/s]


### b. Unzip Model dan Scaler ke Folder `models`

In [ ]:
!unzip models.zip -d models

Archive:  models.zip
  inflating: models/feature_columns.pkl  
  inflating: models/__MACOSX/._feature_columns.pkl  
  inflating: models/feature_scaler.pkl  
  inflating: models/__MACOSX/._feature_scaler.pkl  
  inflating: models/price_scaler.pkl  
  inflating: models/__MACOSX/._price_scaler.pkl  
  inflating: models/random_forest_regressor_model.pkl  
  inflating: models/__MACOSX/._random_forest_regressor_model.pkl  


## 3. Aplikasi Streamlit

### a. Membuat Aplikasi Streamlit

In [ ]:
!kill -9 -f streamlit

/bin/bash: line 1: kill: -f: arguments must be process or job IDs
/bin/bash: line 1: kill: streamlit: arguments must be process or job IDs


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

from babel.numbers import format_currency

st.set_page_config(page_title = "Prediksi Harga Rumah", layout = "centered")
st.title("Aplikasi Prediksi Harga Rumah")

@st.cache_resource
def load_assets():
  try:
    model = joblib.load("models/random_forest_regressor_model.pkl")
    feature_colums = joblib.load("models/feature_columns.pkl")
    feature_scaler = joblib.load("models/standard_scaler.pkl")
    price_scaler = joblib.load("models/price_scaler.pkl")
    return model, feature_colums, feature_scaler, price_scaler
  except:
    return None, None, None, None

# load asset
model, feature_colums, feature_scaler, price_scaler = load_assets()

if model is None:
  st.error("Gagal loading model")
else:
  st.markdown("""
  Masukan detail property di bawah ini untuk mendapatkan estimasi harga.
  Aplikasi ini menggunakan model machine learning untuk memberikan prediksi harga
  """)

  with st.form('prediction_form'):
    st.header('Masukan detail property')

    col1, col2 = st.columns(2)
    with col1:
      area = st.number_input("Luas tanah (m2)", min_values=30.0,
                             max_value=1000.0, value=120.0, step=10.0)
      bedrooms = st.number_input("Jumlah kamar tidur", min_values=1,
                             max_value=10, value=3, step=1)
      garage = st.number_input("Kapasitas garasi (mobil)", min_values=0,
                             max_value=5, value=1, step=1)

    with col2:
      building_area = st.number_input("Luas bangunan (m2)", min_values=30.0,
                             max_value=800.0, value=90.0, step=10.0)
      bathrooms = st.number_input("Jumlah kamar mandi", min_values=1,
                             max_value=8, value=1, step=1)
      city = st.select_box("Kota", ('Jakarta Selatan', 'Jakarta Timur',
                                    'Jakarta Barat', 'Depok', 'Bogor',
                                    'Bekasi', 'Tangerang', 'Tangerang Selatan'))

    submit_button = st.form_submit_button(label="Prediksi")

  if submit_button:
    try:
      input_data = {
          'area': [area],
          'building_area': [building_area],
          'bedrooms': [bedrooms],
          'bathrooms': [bathrooms],
          'city': [city],
          'garage': [garage],
      }

      input_df = pd.DataFrame(input_data)
      input_df[['area', 'building_area']] = feature_scaler.transform(
          input_df[['area', 'building_area']]
      )
      input_df = pd.get_dummies(input_df, columns = ['city', 'bedrooms', 'bathrooms', 'garage'],
                                          prefix = ['City', 'Bedroom', 'Bathroom', 'Garage'])

      input_processed = input_df.reindex(columns=feature_columns, fill_value=0)

      scaled_prediction = model.predict(input_processed)
      original_price = price_scaler.inverse_transform(scaled_prediction.reshape(-1, 1))[0][0] * 1000000
      #st.error(f'{scaled_prediction}')
      original_price = format_currency(original_price, "IDR", locale="id_ID")

      st.success('Prediksi berhasil')
      st.metric(
          label="Estimasi harga prediksi",
          value=original_price
      )
    except Exception as e:
      st.error(f'Terjadi kesalahan')


Overwriting app.py


### b. Menjalankan Aplikasi Streamlit

In [ ]:
!streamlit run app.py &>/dev/null&

### c. Membuat Tunnel Menggunakan Pinggy.io

Tunneling berfungsi agar aplikasi streamlit yang berjalan di Google Colab dapat diakses dari browser

In [ ]:
import time
import sys

!ssh -o StrictHostKeyChecking=no -p 443 -R0:localhost:8501 a.pinggy.io > pinggy_log.txt 2>&1 &

print("   Menunggu URL public...", end="")

time.sleep(3)
found_url = False

for i in range(10):
    print(".", end="")
    try:
        with open('pinggy_log.txt', 'r') as f:
            log_content = f.read()

        if "http:" in log_content and ".pinggy.link" in log_content:
            import re

            urls = re.findall(r'http[s]?://[a-zA-Z0-9.-]+\.pinggy\.link', log_content)

            if urls:
                print("\n\nAPLIKASI SIAP! Klik link di bawah ini:")
                print("🔗 " + urls[0])
                found_url = True
                break
    except:
        pass
    time.sleep(2)

if not found_url:
    print("\n\nURL tidak ditemukan.")
    print("Coba buka file 'pinggy_log.txt' di folder sebelah kiri secara manual.")

   Menunggu URL public....

APLIKASI SIAP! Klik link di bawah ini:
🔗 http://bfdwc-35-238-196-233.a.free.pinggy.link


## 4. Deploy ke Streamlit Cloud

Sebelum melakukan deploy ke Streamlit cloud, kita harus membuat akun github terlebih dahulu, kemudian mengupload project ke github.

### a. Membuat File `requirements.txt`

In [ ]:
%%writefile requirements.txt
streamlit==1.50.0
babel==2.17.0
pandas==2.3.3
joblib==1.5.2
scikit-learn==1.7.2

Writing requirements.txt


### b. Upload ke Github

In [ ]:
import os
from getpass import getpass

print("Masukkan Username GitHub Anda:")
username = input()

print("Masukkan Nama Repository (yang baru dibuat, misal: prediksi-rumah):")
repo_name = input()

print("Masukkan Personal Access Token:")
token = getpass()

email = input("Masukkan Email GitHub Anda: ")
!git config --global user.email "$email"
!git config --global user.name "$username"

if os.path.exists('.git'):
    !rm -rf .git

!git init
!git branch -M main

!git add app.py requirements.txt models

!git commit -m "First commit from Google Colab"

remote_url = f"https://{token}@github.com/{username}/{repo_name}.git"
!git remote add origin $remote_url
!git push -u origin main

print(f"\nUpload berhasil! Cek repository di: https://github.com/{username}/{repo_name}")

Masukkan Username GitHub Anda:
alfanugraha
Masukkan Nama Repository (yang baru dibuat, misal: prediksi-rumah):
prediksi_rumah
Masukkan Personal Access Token:
··········
Masukkan Email GitHub Anda: alfa.nugraha4@gmail.com
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
[main (root-commit) f7ffea7] First commit from Google Colab
 10 files changed, 35 insertions(+)
 create mode 100644 app.py
 create mode 100644 models/__MACOSX/._feature_columns.pkl
 create mode 100644 models/__MACOSX/._feature_scaler.pkl
 crea